# Data Preprocessing Tools

## Importing the libraries



In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Importing the dataset

In [17]:
daturl = 'https://raw.githubusercontent.com/sakuronohana/my_datascience/master/udemy/mlaz/Part%201%20-%20Data%20Preprocessing/Python/Data.csv'

dataset = pd.read_csv(daturl)

### Selecting data

Wenn wir Supervised Learning durchführen, dann besteht der Datensatz meist aus einem **Label** (y, abhängigen Variable / dependent Variable) und den **Merkmalen** (Features, X, unabhängigen Variablen / independent Variables).
Es ist Standard, das in einem Datensatz das Label meist in der hintersten Spalte ist.

Wir importieren nun die die Labels und Features in verschieden Matrixen

In [18]:
X = dataset.iloc[:,:-1].values # Erste Doppelpunkt ist der Range mit Rows und zweiter Doppelpunkt der Range mit Columns. Mit Values erstellen wir eine Matrix mit den Werten ohne Header
y = dataset.iloc[:,-1].values

In [19]:
print('Hier sind die Labels:',y)
print('Hier sind die Features:\n',X)

Hier sind die Labels: ['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']
Hier sind die Features:
 [['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Taking care of missing data

Es kommt des öfteren vor, dass in einem Datensatz Werte fehlen. Die Art und Weise wie man mit fehlenden Daten umgeht ist unterschiedlich und geht von den Werten ersetzen (mit Mean, Median usw.) bis löschen, was bei einem grossen Datensatz meist kein Problem darstellt jedoch bei einem kleinen schon.

Im folgenden Beispiel werden wir die fehlenden Werte mittels arithm. Mittelwert ersetzen. Dazu verwenden wir eines der beliebtesten ML Tools **scikit-learn**. 

In [20]:
# Schauen wir mal zuerst wo wir fehlende Wert haben
dataset.isna()

,Country,Age,Salary,Purchased
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,True,False
5,False,False,False,False
6,False,True,False,False
7,False,False,False,False
8,False,False,False,False
9,False,False,False,False


In [21]:
# Um alle fehlenden Werte in den Features zu ersetzen verwenden wir SimpleImputer
from sklearn.impute import SimpleImputer

# Zuerst ersellen eine Instanz der Klasse SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') # Der erste Wert in der Klammer bedeutet, was wir ersetzen und der zweite Wert mit was

# Nun benötigen wir noch eine Funktion mit welcher wir die Klasse auf unseren Datensatz anwenden können. Hierzu steht die Funktion fit
imputer.fit(X[:,1:3]) #im Datensatz werden wir in allen nummerischen Features nach fehlendne Werten suchen

# Nachfolgend führe wir nun die Transformation des Datensatzes durch
X[:,1:3] = imputer.transform(X[:,1:3]) #Die Änderungen werden direkt im Datensatz durchgeführt

In [22]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Encoding categorical data

Ein Machine Learning Algorithmus kann mit kategorische Werten nicht umgehen, weshalb diese in einem speziellen Verfahren, dass sich Encoding nennt, in nummerische Werte umgewandelt werden müssen.

Das OneHot-Encoding wandelt alle gleiche Features in einen spezifischen Vektor um.


### Encoding the Independent Variable

In [23]:
# Wir Laden nun das entsprechende Modul von Scikit-learn
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Nun erstellen wir eine Klasse und speichern diese direkt in einer Funktion
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])], remainder='passthrough') # Erstes Argument ist der Encoder mit der Spalte, zweites Argument ignoriert die restlichen Spalten
X = np.array(ct.fit_transform(X)) # ML Algorithmen erwarten grundsätzlich immer eine Array weshalb wir die neuen Spalten auch entsprechendn konvertieren müssen

In [26]:
print('Unser kategorischen Fearture Country besteht nun aus 3 Spalten und wurde transformiert in ein für die Machine lesbares Format \n', X)

Unser kategorischen Fearture Country besteht nun aus 3 Spalten und wurde transformiert in ein für die Machine lesbares Format 
 [[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Encoding the Dependent Variable

Nun Kodieren wir noch den Label-Datensatz. Dieses Encoding ist leicht anders als das von X, da wir beim Label nur zwei Zustände (Yes/No) haben. Weshalb wir hier mit dem sogenannten Label Encoding arbeiten können.  

In [29]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() # Erstellen einer Klasse
y = le.fit_transform(y) # Da beim LabelEncoding keine zusätzlichen Spalten hinzugefügt werden muss hier kein Array erstellt werden.

In [30]:
print(y)

[0 1 0 0 1 1 0 1 0 1]


## Splitting the dataset into the Training set and Test set

Nun müssen wir noch die Datensätze X und y in einen Trainings- und Testdatensatz aufteilen. Hierzu hat Scikit Learn ebensfalls ein entsprechendens Tool zur Hand. Die allerseits gängige Grösse des Splits ist 80/20 d.h. 80% Trainingsdaten 20% Testdaten.

An dieser Stelle möchten wir noch eine wichtige Frage klären. Teilen wir einen Datensatz vor oder nach dem Feature Scaling in jeweils ein Trainings- und Testset auf? Die Antwort ist ganz einfach. **Vor dem Feature Scaling**

Der Grund dafür ist eben so einfach wie die Antwort. Wir wollen, dass die Testdaten nicht in der gleichen Art skaliert werden wie die Trainingsdaten. Würde wir den umgekehrten Weg beschreiten, dann könntes es zu einem sogenannten [Data Leakage](https://machinelearningmastery.com/data-leakage-machine-learning/) führen. 

In [31]:
from sklearn.model_selection import train_test_split

'''
Nachfolgend teilen wir die Datensätze in Traing und Test auf, dabei definieren wir alle Variablen X_train usw.
geben in den Argumenten mit, welche Datensatz wir spliten, wie gross der Testset sein wird und mit welchem Random Seed
wir arbeiten.
'''
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 1 )

In [35]:
print(X_train, X_test)
print(y_train,y_test)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]] [[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]
[0 1 0 0 1 1 0 1] [0 1]


## Feature Scaling

Nun kommen wir zu einem anderen wichtigen Thema der Datenvorbereitung - Feature Scaling.

Ziel der Skalierung ist es zu verhindern, dass während der ML Trainingsphase die einen Features nicht zu stark von den anderen Features dominiert bzw. ignoriert werden. Hierbei ist noch zu erwähnen, dass nicht bei allen ML Methoden eine Skalierung der Daten notwendig ist.

Es gibt zwei hauptsächliche Feature Scaling Methoden:

* Standardisierung - Transformiert die Daten in Werte -1 + 1. Diese Methode kann in den meisten der Datensätze angewendet werden. Somit gehört sie zu den beliebtesten Methoden.
* Normalisierung - Transformiert die Daten in Werte von 0 - 1. Diese Methode ist nur dann zulässig, wenn eine Normalverteilung im Datensatz besteht.

In diesem Zusammenhang gibt es noch eine wichtige Frage zu klären. 
**Muss man Feature Scaling auch auf die kodierten kategorischen Features anwenden?** die Antwort auch hier ist ganz einfach - **Nein**. Diese kodierten kategorischen Features sind ja bereits in Dummy-Werten von 0 und 1 umgewandelt worden. Würde wir diese noch einmal umwandeln, dann bekommen wir jedesmal komische Werte, mittels denen wir ja die ursprünglichen Zuordnung nicht mehr machen könnten.

### Bedeutung von fit und transform

Im Zusammenhang mit der Standardskalierung von Trainings- und Testdaten ist es wichtig den Unterschied zwischen der Funktion fit und transform zu wissen:

"fit" berechnet den Mittelwert und den Standard, die für die spätere Skalierung verwendet werden sollen. (nur eine Berechnung), dir wird nichts gegeben .

"transform" verwendet einen zuvor berechneten Mittelwert und Standard, um die Daten automatisch zu skalieren (Mittelwert von allen Werten subtrahieren und dann durch Standard dividieren).

"fit_transform" macht beides gleichzeitig. Sie können dies also mit 1 Codezeile anstelle von 2 tun.

Schauen wir uns das jetzt in der Praxis an:

Für den X-Trainingssatz führen wir "fit_transform" aus, da wir Mittelwert und Standard berechnen und dann zur automatischen Skalierung der Daten verwenden müssen. Für den X- Testsatz haben wir bereits den Mittelwert und den Standard, also machen wir nur den Teil "transformieren".


In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler() # Wir erstellen wie gewohnt eine Klasse


X_train[:,3:] = sc.fit_transform(X_train[:,3:]) #fit holt die erforderlichen Werte wie Mean und SD, Transform macht die Standardisierung

X_test[:,3:] = sc.Transform(X_test[:,3:]) 